## Wilkie et al. 2010
### Rat skeletal muscle

In [20]:
# Excel import
wb = openpyxl.load_workbook('./SourceData/Wilkie_2010/mcp.M110.003129-6.xlsx')
print(wb.get_sheet_names())

# sheet selected
sheet = wb.get_sheet_by_name('Extracted')
_df_wilkie = pd.DataFrame(sheet.values)

# convert to df
df_wilkie = pd.DataFrame(_df_wilkie.iloc[3:, 1])
df_wilkie.columns = ['Gene_name']
df_wilkie = df_wilkie.reset_index(drop=True)
df_wilkie

['Table 1', 'Extracted']


<ipython-input-20-ad279f51e5a2>:2: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  print(wb.get_sheet_names())
<ipython-input-20-ad279f51e5a2>:4: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  sheet = wb.get_sheet_by_name('Extracted')


In [42]:
# uniprot API URL
WEBSITE_API = "https://rest.uniprot.org/uniprotkb/"

# organism id
organism_id_list = {'Homo sapiens': '9606', 'Mus musculus': '10090'}
organism_id = organism_id_list['Homo sapiens']

# find Uniprot Entry and gene names for each gene
for i in range(len(df_wilkie)):
    
    # gene name
    gene = df_wilkie.iloc[i, 0]

    try:
        # get response with gene name the query
        r = get_url(f'{WEBSITE_API}/search?query=gene:{gene}+AND+organism_id:{organism_id}&fields=accession,gene_names')
        result = r.json()['results'][0]
        
        ## get human or mouse entry
        entry_converted = result['primaryAccession']
        ## get gene name and check the names match between human's or mouse's and the given organism's
        gene_obtained = result['genes'][0]['geneName']['value']
    
    except:
        entry_converted = 'Not_found'
    
    # Put the obtained Entry and gene name
    df_wilkie.loc[i, 'Entry_Wilkie'] = entry_converted
    df_wilkie.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    # take a break and go next
    sleep(1)

In [46]:
# Manually check the result
# df_wilkie.head(n=50)
# df_wilkie.tail(n=50)

In [47]:
df_wilkie = df_wilkie.drop(['Gene_name'], axis=1)

In [48]:
df_wilkie.to_csv('./Output/Wilkie_2010.csv', index=False)